# Transfer learning appliqué à la dataset du titanic (Classification binaire)

## Importer les données train et test du titanic

In [1]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(889, 10)
(417, 10)
(889, 1)
(417, 1)


## Importer l'architecture du modèle et les poids

In [2]:
from keras.models import model_from_json

# Charger le fichier json and créer le modèle
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# Charger les poids du modèle
loaded_model.load_weights("model.h5")
print("Le modèle et les poids sont chargés")

Using TensorFlow backend.


Le modèle et les poids sont chargés


## Ajouter une couche d'entrée et figer les poids du modèle entrainé

In [3]:
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential

# Créer un modèle séquentiel avec une couche
model = Sequential()
model.add(Dense(63, input_dim=10, activation='relu', name='dense_0'))
# Ajouter les couches du modèle entrainé
for i in range(len(loaded_model.layers)):
    model.add(loaded_model.layers[i])

# Figer les poids du modèle entrainé
for i in range(1,len(model.layers)):
    model.layers[i].trainable = False

# Compiler le modèle
adam = Adam(lr=0.001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])


In [4]:
# Entrainer le modèle
history = model.fit(X_train, y_train, validation_split=0.20, batch_size=50, epochs=100)

Train on 711 samples, validate on 178 samples
Epoch 1/100
711/711 [==============================] - 0s 491us/step - loss: 3.5033 - acc: 0.3924 - val_loss: 2.6837 - val_acc: 0.3539
Epoch 2/100
711/711 [==============================] - 0s 46us/step - loss: 1.9835 - acc: 0.4149 - val_loss: 1.3588 - val_acc: 0.4045
Epoch 3/100
711/711 [==============================] - 0s 44us/step - loss: 1.0563 - acc: 0.5274 - val_loss: 0.6796 - val_acc: 0.6517
Epoch 4/100
711/711 [==============================] - 0s 44us/step - loss: 0.7063 - acc: 0.6610 - val_loss: 0.5165 - val_acc: 0.7809
Epoch 5/100
711/711 [==============================] - 0s 45us/step - loss: 0.5889 - acc: 0.7314 - val_loss: 0.4536 - val_acc: 0.8202
Epoch 6/100
711/711 [==============================] - 0s 46us/step - loss: 0.5436 - acc: 0.7665 - val_loss: 0.4336 - val_acc: 0.8202
Epoch 7/100
711/711 [==============================] - 0s 46us/step - loss: 0.5584 - acc: 0.7595 - val_loss: 0.4185 - val_acc: 0.8258
Epoch 8/100
711

711/711 [==============================] - 0s 56us/step - loss: 0.4461 - acc: 0.8115 - val_loss: 0.3624 - val_acc: 0.8483
Epoch 62/100
711/711 [==============================] - 0s 48us/step - loss: 0.4347 - acc: 0.8143 - val_loss: 0.3690 - val_acc: 0.8427
Epoch 63/100
711/711 [==============================] - 0s 39us/step - loss: 0.4453 - acc: 0.8101 - val_loss: 0.3643 - val_acc: 0.8539
Epoch 64/100
711/711 [==============================] - 0s 38us/step - loss: 0.4463 - acc: 0.8158 - val_loss: 0.3609 - val_acc: 0.8483
Epoch 65/100
711/711 [==============================] - 0s 37us/step - loss: 0.4262 - acc: 0.8214 - val_loss: 0.3623 - val_acc: 0.8652
Epoch 66/100
711/711 [==============================] - 0s 38us/step - loss: 0.4356 - acc: 0.8186 - val_loss: 0.3617 - val_acc: 0.8652
Epoch 67/100
711/711 [==============================] - 0s 39us/step - loss: 0.4509 - acc: 0.8158 - val_loss: 0.3624 - val_acc: 0.8596
Epoch 68/100
711/711 [==============================] - 0s 39us/step

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_0 (Dense)              (None, 63)                693       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total para

In [6]:
model.evaluate(X_test, y_test) # fonction d'erreur (binary cross entropy) + Accuracy

417/417 [==============================] - 0s 49us/step


[0.27948392162934765, 0.8920863309352518]

In [7]:
from matplotlib import pyplot

# Tracer la courbe d'erreur pendant l'apprentissage
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='validation')
pyplot.legend()
# Tracer la courbe de la précision pendant l'apprentissage
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['acc'], label='train')
pyplot.plot(history.history['val_acc'], label='validation')
pyplot.legend()
pyplot.show()

<Figure size 640x480 with 2 Axes>

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix

# La matrice de confusion
print("La matrice de confusion:\n {}".format(confusion_matrix(y_test, model.predict_classes(X_test))))

La matrice de confusion:
 [[252  13]
 [ 32 120]]


In [9]:
# enregistrer le modèle en format JSON
model_json = model.to_json()
with open("model_titanic.json", "w") as json_file:
    json_file.write(model_json)
# enregistrer les poids en format HDF5
model.save_weights("model_titanic.h5")
print("Saved model to disk")

Saved model to disk
